In [1]:
import tensorflow as tf
import pickle
import os

In [2]:
gen_graph = ["<tf.Variable 'generator/encoder_1/conv/filter:0' shape=(4, 4, 3, 64) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_2/conv/filter:0' shape=(4, 4, 64, 128) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_2/batchnorm/offset:0' shape=(128,) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_2/batchnorm/scale:0' shape=(128,) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_3/conv/filter:0' shape=(4, 4, 128, 256) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_3/batchnorm/offset:0' shape=(256,) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_3/batchnorm/scale:0' shape=(256,) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_4/conv/filter:0' shape=(4, 4, 256, 512) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_4/batchnorm/offset:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_4/batchnorm/scale:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_5/conv/filter:0' shape=(4, 4, 512, 512) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_5/batchnorm/offset:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_5/batchnorm/scale:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_6/conv/filter:0' shape=(4, 4, 512, 512) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_6/batchnorm/offset:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_6/batchnorm/scale:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_7/conv/filter:0' shape=(4, 4, 512, 512) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_7/batchnorm/offset:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_7/batchnorm/scale:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_8/conv/filter:0' shape=(4, 4, 512, 512) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_8/batchnorm/offset:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/encoder_8/batchnorm/scale:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_8/deconv/filter:0' shape=(4, 4, 512, 512) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_8/batchnorm/offset:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_8/batchnorm/scale:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_7/deconv/filter:0' shape=(4, 4, 512, 1024) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_7/batchnorm/offset:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_7/batchnorm/scale:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_6/deconv/filter:0' shape=(4, 4, 512, 1024) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_6/batchnorm/offset:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_6/batchnorm/scale:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_5/deconv/filter:0' shape=(4, 4, 512, 1024) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_5/batchnorm/offset:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_5/batchnorm/scale:0' shape=(512,) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_4/deconv/filter:0' shape=(4, 4, 256, 1024) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_4/batchnorm/offset:0' shape=(256,) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_4/batchnorm/scale:0' shape=(256,) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_3/deconv/filter:0' shape=(4, 4, 128, 512) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_3/batchnorm/offset:0' shape=(128,) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_3/batchnorm/scale:0' shape=(128,) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_2/deconv/filter:0' shape=(4, 4, 64, 256) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_2/batchnorm/offset:0' shape=(64,) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_2/batchnorm/scale:0' shape=(64,) dtype=float32_ref>",
 "<tf.Variable 'generator/decoder_1/deconv/filter:0' shape=(4, 4, 3, 128) dtype=float32_ref>"]

In [ ]:
def preprocess(image):
    with tf.name_scope("preprocess"):
        # [0, 1] => [-1, 1]
        return image * 2 - 1

In [ ]:
def deprocess(image):
    with tf.name_scope("deprocess"):
        # [-1, 1] => [0, 1]
        return (image + 1) / 2

In [ ]:
def check_image(image):
    assertion = tf.assert_equal(tf.shape(image)[-1], 3, message="image must have 3 color channels")
    with tf.control_dependencies([assertion]):
        image = tf.identity(image)

    if image.get_shape().ndims not in (3, 4):
        raise ValueError("image must be either 3 or 4 dimensions")

    # make the last dimension 3 so that you can unstack the colors
    shape = list(image.get_shape())
    shape[-1] = 3
    image.set_shape(shape)
    return image

In [ ]:
def lrelu(x, a):
    with tf.name_scope("lrelu"):
        # adding these together creates the leak part and linear part
        # then cancels them out by subtracting/adding an absolute value term
        # leak: a*x/2 - a*abs(x)/2
        # linear: x/2 + abs(x)/2

        # this block looks like it has 2 inputs on the graph unless we do this
        x = tf.identity(x)
        return (0.5 * (1 + a)) * x + (0.5 * (1 - a)) * tf.abs(x)

In [ ]:
def batchnorm(input):
    with tf.variable_scope("batchnorm"):
        # this block looks like it has 3 inputs on the graph unless we do this
        input = tf.identity(input)

        channels = input.get_shape()[3]
        offset = tf.get_variable("offset", [channels], dtype=tf.float32, initializer=tf.zeros_initializer())
        scale = tf.get_variable("scale", [channels], dtype=tf.float32,
                                initializer=tf.random_normal_initializer(1.0, 0.02))
        mean, variance = tf.nn.moments(input, axes=[0, 1, 2], keep_dims=False)
        variance_epsilon = 1e-5
        normalized = tf.nn.batch_normalization(input, mean, variance, offset, scale, variance_epsilon=variance_epsilon)
        return normalized

In [ ]:
def create_generator(generator_inputs, generator_outputs_channels):
    layers = []

    # encoder_1: [batch, 256, 256, in_channels] => [batch, 128, 128, ngf]
    with tf.variable_scope("encoder_1"):
        output = conv(generator_inputs, a.ngf, stride=2)
        layers.append(output)

    layer_specs = [
        a.ngf * 2,
        a.ngf * 4,
        a.ngf * 8,
        a.ngf * 8,
        a.ngf * 8,
        a.ngf * 8,
        a.ngf * 8,
    ]

    for out_channels in layer_specs:
        with tf.variable_scope("encoder_%d" % (len(layers) + 1)):
            rectified = lrelu(layers[-1], 0.2)
            # [batch, in_height, in_width, in_channels] => [batch, in_height/2, in_width/2, out_channels]
            convolved = conv(rectified, out_channels, stride=2)
            output = batchnorm(convolved)
            layers.append(output)

    layer_specs = [
        (a.ngf * 8, 0.5),
        (a.ngf * 8, 0.5),
        (a.ngf * 8, 0.5),
        (a.ngf * 8, 0.0),
        (a.ngf * 4, 0.0),
        (a.ngf * 2, 0.0),
        (a.ngf, 0.0),
    ]

    num_encoder_layers = len(layers)
    for decoder_layer, (out_channels, dropout) in enumerate(layer_specs):
        skip_layer = num_encoder_layers - decoder_layer - 1
        with tf.variable_scope("decoder_%d" % (skip_layer + 1)):
            if decoder_layer == 0:
                # first decoder layer doesn't have skip connections
                # since it is directly connected to the skip_layer
                input = layers[-1]
            else:
                input = tf.concat([layers[-1], layers[skip_layer]], axis=3)

            rectified = tf.nn.relu(input)
            # [batch, in_height, in_width, in_channels] => [batch, in_height*2, in_width*2, out_channels]
            output = deconv(rectified, out_channels)
            output = batchnorm(output)

            if dropout > 0.0:
                output = tf.nn.dropout(output, keep_prob=1 - dropout)

            layers.append(output)

    # decoder_1: [batch, 128, 128, ngf * 2] => [batch, 256, 256, generator_outputs_channels]
    with tf.variable_scope("decoder_1"):
        input = tf.concat([layers[-1], layers[0]], axis=3)
        rectified = tf.nn.relu(input)
        output = deconv(rectified, generator_outputs_channels)
        output = tf.tanh(output)
        layers.append(output)

    return layers[-1]

In [ ]:
def create_discriminator(discrim_inputs, discrim_targets):
        n_layers = 3
        layers = []

        # 2x [batch, height, width, in_channels] => [batch, height, width, in_channels * 2]
        input = tf.concat([discrim_inputs, discrim_targets], axis=3)

        # layer_1: [batch, 256, 256, in_channels * 2] => [batch, 128, 128, ndf]
        with tf.variable_scope("layer_1"):
            convolved = conv(input, a.ndf, stride=2)
            rectified = lrelu(convolved, 0.2)
            layers.append(rectified)

        # layer_2: [batch, 128, 128, ndf] => [batch, 64, 64, ndf * 2]
        # layer_3: [batch, 64, 64, ndf * 2] => [batch, 32, 32, ndf * 4]
        # layer_4: [batch, 32, 32, ndf * 4] => [batch, 31, 31, ndf * 8]
        for i in range(n_layers):
            with tf.variable_scope("layer_%d" % (len(layers) + 1)):
                out_channels = a.ndf * min(2 ** (i + 1), 8)
                stride = 1 if i == n_layers - 1 else 2  # last layer here has stride 1
                convolved = conv(layers[-1], out_channels, stride=stride)
                normalized = batchnorm(convolved)
                rectified = lrelu(normalized, 0.2)
                layers.append(rectified)

        # layer_5: [batch, 31, 31, ndf * 8] => [batch, 30, 30, 1]
        with tf.variable_scope("layer_%d" % (len(layers) + 1)):
            convolved = conv(rectified, out_channels=1, stride=1)
            output = tf.sigmoid(convolved)
            layers.append(output)

        return layers[-1]

In [ ]:
def sum_tv_loss(image):
        loss_y = tf.nn.l2_loss(image[:, 1:, :, :] - image[:, :-1, :, :])
        loss_x = tf.nn.l2_loss(image[:, :, 1:, :] - image[:, :, :-1, :])
        loss = 2 * (loss_y + loss_x)
        loss = tf.cast(loss, tf.float32)
        return loss

In [ ]:
def feature_loss(image , vgg):
        vgg.build(image)
        loss = vgg.conv3_3
        return loss

In [ ]:
with tf.variable_scope("generator") as scope:
        out_channels = int(targets.get_shape()[-1])
        outputs = create_generator(inputs, out_channels)

In [ ]:
def create_model(inputs, targets, net1,net2):
    with tf.variable_scope("generator") as scope:
        out_channels = int(targets.get_shape()[-1])
        outputs = create_generator(inputs, out_channels)

    # create two copies of discriminator, one for real pairs and one for fake pairs
    # they share the same underlying variables
    with tf.name_scope("real_discriminator"):
        with tf.variable_scope("discriminator"):
            # 2x [batch, height, width, channels] => [batch, 30, 30, 1]
            predict_real = create_discriminator(inputs, targets)

    with tf.name_scope("fake_discriminator"):
        with tf.variable_scope("discriminator", reuse=True):
            # 2x [batch, height, width, channels] => [batch, 30, 30, 1]
            predict_fake = create_discriminator(inputs, outputs)

    with tf.name_scope("discriminator_loss"):
        # minimizing -tf.log will try to get inputs to 1
        # predict_real => 1
        # predict_fake => 0
        discrim_loss = tf.reduce_mean(predict_fake-predict_real)
        #  with tf.name_scope(""):
    with tf.name_scope("generator_loss"):
        # predict_fake => 1
        # abs(targets - outputs) => 0
        gen_loss_GAN = -tf.reduce_mean(predict_fake)
        gen_loss_L1 = tf.reduce_mean(tf.abs(targets - outputs))
        gen_loss_tv = tf.reduce_mean(tf.sqrt(tf.nn.l2_loss(sum_tv_loss(outputs))))
        gen_loss_f = tf.reduce_mean(tf.sqrt(tf.nn.l2_loss(feature_loss(targets,net1) - feature_loss(outputs,net2))))
        gen_loss = gen_loss_GAN * a.gan_weight + gen_loss_L1 * a.l1_weight + gen_loss_tv * a.tv_weight + gen_loss_f * a.f_weight

    with tf.name_scope("discriminator_train"):
        discrim_tvars = [var for var in tf.trainable_variables() if var.name.startswith("discriminator")]
        discrim_optim = tf.train.RMSPropOptimizer(a.lr, a.beta1)
        discrim_grads_and_vars = discrim_optim.compute_gradients(discrim_loss, var_list=discrim_tvars)
        discrim_train = discrim_optim.apply_gradients(discrim_grads_and_vars)
        clip_vars = [tf.assign(var, tf.clip_by_value(var, -0.02, 0.02)) for var in discrim_tvars]
        tuple_vars = tf.tuple(clip_vars, control_inputs=[discrim_train])


    with tf.name_scope("generator_train"):
        with tf.control_dependencies(tuple_vars):
            gen_tvars = [var for var in tf.trainable_variables() if var.name.startswith("generator")]
            gen_optim = tf.train.RMSPropOptimizer(a.lr, a.beta1)
            gen_grads_and_vars = gen_optim.compute_gradients(gen_loss, var_list=gen_tvars)
            gen_train = gen_optim.apply_gradients(gen_grads_and_vars)

    ema = tf.train.ExponentialMovingAverage(decay=0.99)
    update_losses = ema.apply([discrim_loss, gen_loss_GAN, gen_loss_L1, gen_loss_tv, gen_loss_f])

    global_step = tf.contrib.framework.get_or_create_global_step()
    incr_global_step = tf.assign(global_step, global_step + 1)

    return Model(
        predict_real=predict_real,
        predict_fake=predict_fake,
        discrim_loss=ema.average(discrim_loss),
        discrim_grads_and_vars=discrim_grads_and_vars,
        gen_loss_GAN=ema.average(gen_loss_GAN),
        gen_loss_L1=ema.average(gen_loss_L1),
        gen_loss_tv=ema.average(gen_loss_tv),
        gen_loss_f=ema.average(gen_loss_f),
        gen_grads_and_vars=gen_grads_and_vars,
        outputs=outputs,
        train=tf.group(update_losses, incr_global_step, gen_train),
    )

In [9]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Activation, Conv2D, Conv2DTranspose, BatchNormalization, Dropout, LeakyReLU

In [8]:
def create_generator():
    lrelu = LeakyReLU(alpha = 0.2)
    inputs = Input(shape = (256, 256, 1))
    
    encoder_1 = Conv2D(32, (4, 4), (2, 2), padding = 'same')(inputs)
    
    encoder_2 = Conv2D(64, (4, 4), (2, 2), activation = lrelu, padding = 'same')(encoder_1)
    encoder_2 = BatchNormalization()(encoder_2)
    
    encoder_3 = Conv2D(128, (4, 4), (2, 2), activation = lrelu, padding = 'same')(encoder_2)
    encoder_3 = BatchNormalization()(encoder_3)
    
    encoder_4 = Conv2D(256, (4, 4), (2, 2), activation = lrelu, padding = 'same')(encoder_3)
    encoder_4 = BatchNormalization()(encoder_4)
    
    encoder_5 = Conv2D(256, (4, 4), (2, 2), activation = lrelu, padding = 'same')(encoder_4)
    encoder_5 = BatchNormalization()(encoder_5)
    
    encoder_6 = Conv2D(256, (4, 4), (2, 2), activation = lrelu, padding = 'same')(encoder_5)
    encoder_6 = BatchNormalization()(encoder_6)
    
    encoder_7 = Conv2D(256, (4, 4), (2, 2), activation = lrelu, padding = 'same')(encoder_6)
    encoder_7 = BatchNormalization()(encoder_7)
    
    encoder_8 = Conv2D(256, (4, 4), (2, 2), activation = lrelu, padding = 'same')(encoder_7)
    encoder_8 = BatchNormalization()(encoder_8)
    
    decoder_8 = Conv2DTranspose(256, (4, 4), (2, 2), activation = 'relu', padding = 'same')(encoder_8)
    decoder_8 = BatchNormalization()(decoder_8)
    decoder_8 = Dropout(0.5)(decoder_8)
    
    decoder_7 = Conv2DTranspose(256, (4, 4), (2, 2), activation = 'relu', padding = 'same')\
                        (concatenate([decoder_8, encoder_7], axis = -1))
    decoder_7 = BatchNormalization()(decoder_7)
    decoder_7 = Dropout(0.5)(decoder_7)
    
    decoder_6 = Conv2DTranspose(256, (4, 4), (2, 2), activation = 'relu', padding = 'same')\
                        (concatenate([decoder_7, encoder_6], axis = -1))
    decoder_6 = BatchNormalization()(decoder_6)
    decoder_6 = Dropout(0.5)(decoder_6)
    
    decoder_5 = Conv2DTranspose(256, (4, 4), (2, 2), activation = 'relu', padding = 'same')\
                        (concatenate([decoder_6, encoder_5], axis = -1))
    decoder_5 = BatchNormalization()(decoder_5)
    
    decoder_4 = Conv2DTranspose(128, (4, 4), (2, 2), activation = 'relu', padding = 'same')\
                        (concatenate([decoder_5, encoder_4], axis = -1))
    decoder_4 = BatchNormalization()(decoder_4)
    
    decoder_3 = Conv2DTranspose(64, (4, 4), (2, 2), activation = 'relu', padding = 'same')\
                        (concatenate([decoder_4, encoder_3], axis = -1))
    decoder_3 = BatchNormalization()(decoder_3)
    
    decoder_2 = Conv2DTranspose(32, (4, 4), (2, 2), activation = 'relu', padding = 'same')\
                        (concatenate([decoder_3, encoder_2], axis = -1))
    decoder_2 = BatchNormalization()(decoder_2)
    
    decoder_1 = Conv2DTranspose(1, (4, 4), (2, 2), activation = 'relu', padding = 'same')\
                        (concatenate([decoder_2, encoder_1], axis = -1))
    
    outputs = Activation('tanh')(decoder_1)
    
    model = Model(inputs = inputs, outputs = outputs)
    
    return model

In [ ]:
def create_discriminator():
    inputs = Input(shape = (256, 256, 2))
    lrelu = LeakyReLU(alpha = 0.2)
    
    layer_1 = Conv2D(32, (4, 4), (2, 2), activation = lrelu, padding = 'same')(inputs)
    
    layer_2 = Conv2D(64, (4, 4), (2, 2), activation = lrelu, padding = 'same')(layer_1)
    layer_2 = BatchNormalization()(layer_2)
    
    layer_3 = Conv2D(128, (4, 4), (2, 2), activation = lrelu, padding = 'same')(layer_2)
    layer_3 = BatchNormalization()(layer_3)
    
    layer_4 = Conv2D(256, (4, 4), (1, 1), activation = lrelu, padding = 'same')(layer_3)
    layer_4 = BatchNormalization()(layer_4)
    
    layer_4 = Conv2D(1, (4, 4), (1, 1), activation = lrelu, padding = 'same')(layer_4)
    
    outputs = Activation('sigmoid')(layer_4)
    
    model = Model(inputs = inputs, outputs = outputs)
    
    return model

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, *args, **kwargs):
        super(MyModel, self).__init__(*args, **kwargs)
        self.loss_tracker = tf.keras.metrics.Mean(name='loss')

    def compute_loss(self, x, y, y_pred, sample_weight):
        loss = tf.reduce_mean(tf.math.squared_difference(y_pred, y))
        loss += tf.add_n(self.losses)
        self.loss_tracker.update_state(loss)
        return loss

    def reset_metrics(self):
        self.loss_tracker.reset_states()

    @property
    def metrics(self):
        return [self.loss_tracker]

In [ ]:
def discriminator_loss(predict_real, predict_fake):
    return tf.reduce_mean(predict_fake - predict_real)

In [ ]:
def generator_loss(predict_fake, targets, outputs, net1, net2):
    gen_loss_GAN = -tf.reduce_mean(predict_fake)
    gen_loss_L1 = tf.reduce_mean(tf.abs(targets - outputs))
    gen_loss_tv = tf.reduce_mean(tf.sqrt(tf.nn.l2_loss(sum_tv_loss(outputs))))
    gen_loss_f = tf.reduce_mean(tf.sqrt(tf.nn.l2_loss(feature_loss(targets,net1) - feature_loss(outputs,net2))))
    gen_loss = gen_loss_GAN + (gen_loss_L1 * 10) + (gen_loss_tv * 1e-5) + (gen_loss_f * 1e-4)
    
    return gen_loss

In [ ]:
generator = create_generator()
discriminator = create_discriminator()

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    
    outputs = create_generator(inputs, out_channels)
    
    
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def conv(batch_input, out_channels, stride):
    with tf.variable_scope("conv"):
        in_channels = batch_input.get_shape()[3]
        filter = tf.get_variable("filter", [4, 4, in_channels, out_channels], dtype=tf.float32,
                                 initializer=tf.random_normal_initializer(0, 0.02))
        # [batch, in_height, in_width, in_channels], [filter_width, filter_height, in_channels, out_channels]
        #     => [batch, out_height, out_width, out_channels]
        padded_input = tf.pad(batch_input, [[0, 0], [1, 1], [1, 1], [0, 0]], mode="CONSTANT")
        conv = tf.nn.conv2d(padded_input, filter, [1, stride, stride, 1], padding="VALID")
        return conv

In [ ]:
def deconv(batch_input, out_channels):
    with tf.variable_scope("deconv"):
        batch, in_height, in_width, in_channels = [int(d) for d in batch_input.get_shape()]
        filter = tf.get_variable("filter", [4, 4, out_channels, in_channels], dtype=tf.float32,
                                 initializer=tf.random_normal_initializer(0, 0.02))
        # [batch, in_height, in_width, in_channels], [filter_width, filter_height, out_channels, in_channels]
        #     => [batch, out_height, out_width, out_channels]
        conv = tf.nn.conv2d_transpose(batch_input, filter, [batch, in_height * 2, in_width * 2, out_channels],
                                      [1, 2, 2, 1], padding="SAME")
        return conv